# Домашнее задание №1

В первом ДЗ мы создаём и обустраиваем git-репозиторий, который будем использовать в дальнейшем, загружаем данные о ц.б. SNP500 и криптовалютах, настраиваем автоматическое отображение графиков котировок и частично проводим предобработку и анализ данных. Для того, чтобы всё сделать правильно, я создал небольшой чек-лист ДЗ1, который Вы видите ниже:

~~1. **Создать git-репозиторий**, где будет храниться исходный код вашего проекта. Если вы используете приватный репозиторий – дайте преподавателям курса доступ к нему, для возможности проверки ДЗ.~~ *(Использовал публичный репозиторий)*

~~2.  **Добавить файл лицензии**, который отражает ваш взгляд на конфиденциальность информации, которую вы подготовите в рамках данного курса.~~ \
\
3. **Создать код на Python**, который загрузит на ваш локальный компьютер данные о:\
    a) котировках ценных бумаг из списка **SnP500**; \
    b) котировки **криптовалют** (BTC, ETH, SOL, XRP). \
\
4. Поскольку вам предстоит много работать с ними в дальнейшем, подготовьте **автоматическое отображение графиков** текущей ситуации. \
\
5.  a) Проверьте нет ли в данных **пропусков или ошибок**; \
    b) Проанализируйте **выбросы**; \
    c) Оцените, на самом ли деле это **выбросы или реальные данные**, с которыми предстоит работать. 

*Примечание: поскольку первые два шага делаются вне этого ноутбука, нумерация разделом начинается с цифры "3"*

## 0: Импорты

In [ ]:
##%pip install pandas
##%pip install requests
##%pip install bs4
#%pip install yfinance --upgrade --no-cache-dir
#%pip install plotly
#%pip install nbformat --upgrade

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly
import yfinance as yf

## 3: Загрузка данных

### 3.a: Загрузка данных о компаниях SNP500

In [3]:
url = 'https://en.m.wikipedia.org/wiki/List_of_S%26P_500_companies' # ссылка на страницу Википедии с тикерами всех компаний индекса SNP500

# собираю тикеры всех компаний из SNP500 в список tickers_snp
tickers_df = pd.read_html(url, attrs={'id': 'constituents'}, index_col='Symbol')[0]
tickers_snp = tickers_df.index.to_list()

*Примечание: я нашёл, как загрузить все тикеры акций SNP500, и реализовал выше; но решил в проекте использовать только 5 с наибольшей рыночной капитализцией (а значит, с самым большим весом в индексе), поскольку работать с данными придётся вручную*

In [4]:
tickers_snp_top5 = ["AAPL", "NVDA", "MSFT", "AMZN", "GOOG"]

In [ ]:
# с помощью yfinance скачиваю данные о топ-5 компаниях индекса SNP500 и кладу их в датафрейм df_snp
df_snp = yf.download(tickers=tickers_snp_top5,
                     start="2023-03-05",
                     end="2024-11-26",
                     interval="1h",
                     group_by="ticker")
df_snp.index = pd.to_datetime(df_snp.index)

[*********************100%***********************]  5 of 5 completed


In [6]:
# отбрасываю колонку adjusted close, содержащую цену закрытия, модифицированную с учётом дивидендов и прочих выплат
df_snp.drop("Adj Close", inplace=True, axis=1, level=1)

### 3.b: Загрузка данных о криптовалютах (BTC, ETH, SOL, XPR)

In [7]:
tickers_ccur = ["BTC-USD", "ETH-USD", "SOL-USD", "XPR-USD"] # список с тикерами криптовалют

In [ ]:
# с помощью yfinance скачиваю данные о криптовалютах и кладу их в датафрейм df_ccur
df_ccur = yf.download(tickers=tickers_ccur,
                      start="2023-03-05",
                      end="2024-11-26",
                      interval="1h",
                      group_by="ticker")
df_ccur.index = pd.to_datetime(df_ccur.index)

[*********************100%***********************]  4 of 4 completed


In [9]:
# отбрасываю колонку adjusted close, содержащую цену закрытия, модифицированную с учётом дивидендов и прочих выплат
df_ccur.drop("Adj Close", inplace=True, axis=1, level=1)

In [28]:
# создаю простые графики с ценами закрытия для Apple и BTC, чтобы удостовериться, что загрузка данных прошла успешно и
# при создании функции-загрузчика им можно пользоваться

fig = go.Figure(layout={"margin" : {"b" : 10, "l" : 5, "t" : 10}})

fig.add_trace(go.Scatter(x=df_snp.index,
                         y=df_snp[("AAPL", "Close")],
                         mode="lines",
                         name="Цена закрытия Apple",
                         showlegend=True,
                         line={"color" : "#171CC9"}))
fig.show()

fig.data = []
fig.add_trace(go.Scatter(x=df_ccur.index,
                         y=df_ccur[("BTC-USD", "Close")],
                         mode="lines",
                         name="Цена закрытия BTC",
                         showlegend=True,
                         line={"color" : "#814E1D"}))
fig.show()